In [ ]:
import os

# Ruta base en Google Drive
base_dir = '/content/drive/MyDrive/sis421/dataset'

# Rutas de imágenes y etiquetas
images_dir = os.path.join(base_dir, 'images')
annotations_file = os.path.join(base_dir, 'annotations.json')


In [ ]:
import json

# Cargar el archivo de anotaciones
with open(annotations_file, 'r') as f:
    annotations = json.load(f)



In [ ]:
from PIL import Image
import torch

class MedicinalPlantsDataset(torch.utils.data.Dataset):
    def __init__(self, annotations, images_dir, transforms=None):
        self.annotations = annotations
        self.images_dir = images_dir
        self.transforms = transforms

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        img_path = os.path.join(self.images_dir, annotation["file_name"])
        img = Image.open(img_path).convert("RGB")

        # Bounding boxes y etiquetas
        boxes = torch.tensor(annotation["boxes"], dtype=torch.float32)
        labels = torch.tensor(annotation["labels"], dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        if self.transforms:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.annotations)


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = get_model(num_classes)
model.to(device)


In [ ]:
# Dividir el dataset
from sklearn.model_selection import train_test_split

train_annotations, val_annotations = train_test_split(annotations, test_size=0.2, random_state=42)

train_dataset = MedicinalPlantsDataset(train_annotations, images_dir)
val_dataset = MedicinalPlantsDataset(val_annotations, images_dir)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Entrenar
num_epochs = 10
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, targets in tqdm(train_loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Adelante y cálculo de pérdida
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        epoch_loss += losses.item()

        # Atrás y optimización
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


In [ ]:
output_path = '/content/drive/MyDrive/medicinal_plants_detector.pth'
torch.save(model.state_dict(), output_path)
print(f"Modelo guardado en {output_path}")


In [ ]:
model.eval()
test_image_path = '/content/drive/MyDrive/sis421/dataset/images/test_image.jpg'

with torch.no_grad():
    img = Image.open(test_image_path).convert("RGB")
    transform = torchvision.transforms.ToTensor()
    img_tensor = transform(img).to(device)

    output = model([img_tensor])  # Predicción
    print(output)  # Muestra las detecciones


In [ ]:
import matplotlib.pyplot as plt
import cv2

# Dibujar las detecciones
def draw_boxes(image_path, boxes):
    img = cv2.imread(image_path)
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return img

# Visualizar
img_with_boxes = draw_boxes(test_image_path, output[0]['boxes'].cpu().numpy())
plt.imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
plt.show()
